<a href="https://colab.research.google.com/github/pabherher19/blank-app/blob/main/Predicci%C3%B3n_VAD_VDL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Actualizar repositorios e instalar paquetes necesarios
!apt-get update
!apt-get install -y xvfb python3-tk
!pip install pyvirtualdisplay
!pip install ttkthemes
!pip install Pillow
!pip install matplotlib seaborn

# Instalación de las dependencias necesarias
!pip install numpy pandas matplotlib seaborn ipywidgets

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [ ]:
%%writefile vad_calculator.py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML
import ipywidgets as widgets

# Configuración de estilo
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_context("notebook", font_scale=1.2)

class AirwayDifficultyPredictor:
    def __init__(self):
        # Medias y desviaciones estándar estimadas para normalización
        self.means = {
            'Superficie lingual cm2': 24.32,
            'Distancia Piel a Epiglotis': 2.10,
            'Grosor de la lengua': 5.82,
            'Distancia Piel a Hueso Hioides': 1.11,
            'EDAD': 53.5,
            'IMC kg/m2': 26.85
        }

        self.stds = {
            'Superficie lingual cm2': 4.66,
            'Distancia Piel a Epiglotis': 0.47,
            'Grosor de la lengua': 0.78,
            'Distancia Piel a Hueso Hioides': 0.41,
            'EDAD': 17.75,
            'IMC kg/m2': 3.98
        }

        # Importancia de variables según el estudio
        self.importances = {
            'Superficie lingual cm2': 0.370635,
            'Distancia Piel a Epiglotis': 0.222797,
            'Grosor de la lengua': 0.159394,
            'Distancia Piel a Hueso Hioides': 0.117933,
            'IMC kg/m2': 0.074609,
            'EDAD': 0.054630
        }

        # Rangos aceptables para cada variable
        self.ranges = {
            'Superficie lingual cm2': (15.0, 33.63),
            'Distancia Piel a Epiglotis': (1.17, 3.03),
            'Grosor de la lengua': (4.27, 7.37),
            'Distancia Piel a Hueso Hioides': (0.3, 1.92),
            'EDAD': (18, 89),
            'IMC kg/m2': (18.9, 34.8)
        }

    def normalize_input(self, input_data):
        """Normaliza los datos de entrada usando z-score"""
        normalized = {}
        for key, value in input_data.items():
            normalized[key] = (value - self.means[key]) / self.stds[key]
        return normalized

    def validate_input(self, input_data):
        """Valida que los valores estén dentro de los rangos aceptables"""
        invalid_fields = []
        for key, (min_val, max_val) in self.ranges.items():
            if not min_val <= input_data[key] <= max_val:
                invalid_fields.append((key, min_val, max_val))
        return invalid_fields

    def predict(self, input_data):
        """Predice la dificultad de vía aérea basada en los datos de entrada"""
        # Normalizar los datos
        normalized_data = self.normalize_input(input_data)

        # Calcular score ponderado basado en importancias
        score = 0
        contribution = {}  # Para almacenar la contribución de cada variable

        for feature, importance in self.importances.items():
            # Calcular contribución según el tipo de variable
            if feature in ['Superficie lingual cm2', 'Grosor de la lengua', 'IMC kg/m2']:
                # Relación directa con dificultad
                contrib = normalized_data[feature] * importance
                score += contrib
            elif feature in ['Distancia Piel a Epiglotis', 'Distancia Piel a Hueso Hioides']:
                # Relación inversa (menor distancia, mayor dificultad)
                contrib = -normalized_data[feature] * importance
                score += contrib
            else:  # EDAD
                # Relación directa pero con menor peso
                contrib = normalized_data[feature] * importance * 0.5
                score += contrib

            contribution[feature] = contrib

        # Determinar clase según el score
        if score < -0.5:
            prediction = 0  # Fácil
            confidence = min(0.95, 0.75 - score)  # Mayor confianza mientras más negativo sea el score
        elif score < 1.0:
            prediction = 1  # Difícil
            # Calcular confianza según proximidad al centro del rango
            # Más cerca del centro = mayor confianza
            mid_range = 0.25  # Punto medio aproximado del rango (-0.5 a 1.0)
            distance_from_mid = abs(score - mid_range)
            confidence = max(0.65, 0.85 - distance_from_mid*0.2)
        else:
            prediction = 2  # Muy difícil
            confidence = min(0.95, 0.6 + score*0.15)  # Mayor confianza mientras más alto sea el score

        # Probabilidades aproximadas (suma = 1)
        if prediction == 0:
            probabilities = [confidence, (1-confidence)*0.8, (1-confidence)*0.2]
        elif prediction == 1:
            probabilities = [(1-confidence)*0.4, confidence, (1-confidence)*0.6]
        else:
            probabilities = [(1-confidence)*0.1, (1-confidence)*0.9, confidence]

        return {
            'prediction': prediction,
            'confidence': confidence,
            'probabilities': probabilities,
            'score': score,
            'contributions': contribution
        }

# Función para obtener un nombre amigable para la variable
def get_friendly_name(var_name):
    """Devuelve un nombre más amigable para la variable"""
    friendly_names = {
        'Superficie lingual cm2': 'Superficie lingual (cm²)',
        'Distancia Piel a Epiglotis': 'Distancia piel a epiglotis (cm)',
        'Grosor de la lengua': 'Grosor de la lengua (cm)',
        'Distancia Piel a Hueso Hioides': 'Distancia piel a hioides (cm)',
        'EDAD': 'Edad (años)',
        'IMC kg/m2': 'IMC (kg/m²)'
    }
    return friendly_names.get(var_name, var_name)

# Función para crear gráfico de importancia de variables
def plot_feature_importance(predictor):
    """Visualiza la importancia de las variables"""
    plt.figure(figsize=(10, 6))

    # Datos para el gráfico
    features = list(predictor.importances.keys())
    importances = list(predictor.importances.values())

    # Ordenar por importancia
    sorted_indices = np.argsort(importances)
    sorted_features = [get_friendly_name(features[i]) for i in sorted_indices]
    sorted_importances = [importances[i] for i in sorted_indices]

    # Crear barras horizontales con gradiente
    colors = plt.cm.Blues(np.linspace(0.4, 0.9, len(features)))
    y_pos = np.arange(len(features))

    # Graficar barras horizontales
    bars = plt.barh(y_pos, sorted_importances, color=colors)

    # Etiquetas y aspecto
    plt.yticks(y_pos, sorted_features)
    plt.xlabel('Importancia Relativa')
    plt.title('Importancia de Variables en la Predicción', fontweight='bold')

    # Añadir valores en las barras
    for i, v in enumerate(sorted_importances):
        plt.text(v + 0.01, i, f'{v:.2f}', va='center', fontsize=9)

    # Ajustar diseño
    plt.tight_layout()
    plt.show()

# Función para graficar contribuciones
def plot_contributions(result):
    """Visualiza la contribución de cada variable al score final"""
    plt.figure(figsize=(10, 6))

    # Obtener contribuciones
    contributions = result['contributions']

    # Ordenar por magnitud absoluta de contribución
    features = list(contributions.keys())
    contrib_values = list(contributions.values())

    # Ordenar por contribución absoluta
    sorted_indices = np.argsort([abs(c) for c in contrib_values])
    sorted_features = [get_friendly_name(features[i]) for i in sorted_indices]
    sorted_contrib = [contrib_values[i] for i in sorted_indices]

    # Colores según si contribuye positiva o negativamente
    colors = ['#4CAF50' if c < 0 else '#E53935' for c in sorted_contrib]
    y_pos = np.arange(len(features))

    # Graficar barras horizontales
    bars = plt.barh(y_pos, sorted_contrib, color=colors)

    # Línea vertical en cero
    plt.axvline(x=0, color='black', linestyle='-', linewidth=0.5)

    # Etiquetas y aspecto
    plt.yticks(y_pos, sorted_features)
    plt.xlabel('Contribución al Score')
    plt.title('Influencia de cada Variable en este Paciente', fontweight='bold')

    # Añadir valores en las barras
    for i, v in enumerate(sorted_contrib):
        plt.text(
            v + (0.01 if v >= 0 else -0.01),
            i,
            f'{v:.2f}',
            va='center',
            ha='left' if v >= 0 else 'right',
            fontsize=9
        )

    # Añadir notas explicativas
    plt.text(
        0.98, 0.02,
        'Verde: Reduce dificultad\nRojo: Aumenta dificultad',
        transform=plt.gca().transAxes,
        ha='right', va='bottom',
        fontsize=9,
        bbox=dict(facecolor='white', alpha=0.8, edgecolor='none')
    )

    # Ajustar diseño
    plt.tight_layout()
    plt.show()

# Función para graficar probabilidades
def plot_probabilities(result):
    """Visualiza las probabilidades de cada clase"""
    plt.figure(figsize=(8, 5))

    # Datos para el gráfico
    labels = ['Fácil', 'Difícil', 'Muy difícil']
    probabilities = result['probabilities']

    # Colores para cada clase
    colors = ['#4CAF50', '#FF9800', '#E53935']

    # Graficar barras verticales
    bars = plt.bar(labels, probabilities, color=colors, width=0.6)

    # Resaltar la predicción principal
    prediction_idx = result['prediction']
    bars[prediction_idx].set_edgecolor('black')
    bars[prediction_idx].set_linewidth(2)

    # Etiquetas y aspecto
    plt.ylabel('Probabilidad')
    plt.ylim(0, 1)
    plt.title('Probabilidad de cada Clase', fontweight='bold')

    # Añadir porcentajes sobre las barras
    for i, v in enumerate(probabilities):
        plt.text(
            i, v + 0.02,
            f'{v*100:.1f}%',
            ha='center', va='bottom',
            fontsize=10
        )

    # Ajustar diseño
    plt.tight_layout()
    plt.show()

# Crear una interfaz interactiva con widgets de ipywidgets
def create_interactive_calculator():
    # Crear el predictor
    predictor = AirwayDifficultyPredictor()

    # Crear widgets para cada parámetro
    superficie_lingual = widgets.FloatSlider(
        value=24.0,
        min=15.0,
        max=33.63,
        step=0.1,
        description='Superficie lingual (cm²):',
        style={'description_width': 'initial'},
        layout={'width': '100%'}
    )

    distancia_piel_epiglotis = widgets.FloatSlider(
        value=2.0,
        min=1.17,
        max=3.03,
        step=0.01,
        description='Distancia piel a epiglotis (cm):',
        style={'description_width': 'initial'},
        layout={'width': '100%'}
    )

    grosor_lengua = widgets.FloatSlider(
        value=6.0,
        min=4.27,
        max=7.37,
        step=0.01,
        description='Grosor de la lengua (cm):',
        style={'description_width': 'initial'},
        layout={'width': '100%'}
    )

    distancia_piel_hioides = widgets.FloatSlider(
        value=1.0,
        min=0.3,
        max=1.92,
        step=0.01,
        description='Distancia piel a hioides (cm):',
        style={'description_width': 'initial'},
        layout={'width': '100%'}
    )

    edad = widgets.IntSlider(
        value=55,
        min=18,
        max=89,
        step=1,
        description='Edad (años):',
        style={'description_width': 'initial'},
        layout={'width': '100%'}
    )

    imc = widgets.FloatSlider(
        value=25.0,
        min=18.9,
        max=34.8,
        step=0.1,
        description='IMC (kg/m²):',
        style={'description_width': 'initial'},
        layout={'width': '100%'}
    )

    # Crear botón de predicción
    predict_button = widgets.Button(
        description='Predecir',
        button_style='primary',
        layout={'width': '200px', 'margin': '10px 0'}
    )

    # Crear área para mostrar resultados
    result_html = widgets.HTML(
        value='<h3>Resultado de la predicción</h3><p>Utilice el botón "Predecir" para obtener un resultado.</p>'
    )

    # Función para actualizar el resultado cuando se hace clic en el botón
    def on_predict_button_clicked(b):
        # Recopilar los valores de entrada
        input_data = {
            'Superficie lingual cm2': superficie_lingual.value,
            'Distancia Piel a Epiglotis': distancia_piel_epiglotis.value,
            'Grosor de la lengua': grosor_lengua.value,
            'Distancia Piel a Hueso Hioides': distancia_piel_hioides.value,
            'EDAD': edad.value,
            'IMC kg/m2': imc.value
        }

        # Validar datos
        invalid_fields = predictor.validate_input(input_data)
        if invalid_fields:
            error_msg = "<h3 style='color: #E53935;'>Valores fuera de rango:</h3><ul>"
            for field, min_val, max_val in invalid_fields:
                error_msg += f"<li>{get_friendly_name(field)}: debe estar entre {min_val} y {max_val}</li>"
            error_msg += "</ul>"
            result_html.value = error_msg
            return

        # Realizar la predicción
        result = predictor.predict(input_data)

        # Determinar el texto y color según la predicción
        difficulty_labels = {
            0: ("Vía aérea fácil", "#4CAF50"),  # Verde
            1: ("Vía aérea difícil (frova/guía flexible)", "#FF9800"),  # Naranja
            2: ("Vía aérea muy difícil (pala hiperangulada/otro videolaringoscopio)", "#E53935")  # Rojo
        }

        prediction = result['prediction']
        label, color = difficulty_labels[prediction]

        # Actualizar el texto del resultado
        confidence_pct = int(result['confidence'] * 100)
        result_html.value = f"""
        <h2

Overwriting vad_calculator.py


In [ ]:
# Importar el módulo y ejecutar la calculadora
from vad_calculator import create_interactive_calculator

# Crear y mostrar la calculadora interactiva
create_interactive_calculator()

SyntaxError: unterminated triple-quoted string literal (detected at line 383) (vad_calculator.py, line 382)

In [ ]:
!ls -la


total 36
drwxr-xr-x 1 root root  4096 Mar  1 11:23 .
drwxr-xr-x 1 root root  4096 Mar  1 11:20 ..
drwxr-xr-x 4 root root  4096 Feb 27 14:21 .config
drwx------ 7 root root  4096 Mar  1 11:22 drive
drwxr-xr-x 1 root root  4096 Feb 27 14:22 sample_data
-rw-r--r-- 1 root root 13295 Mar  1 11:24 vad_calculator.py


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML
import ipywidgets as widgets

# Configuración de estilo
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_context("notebook", font_scale=1.2)

# Clase para predecir la dificultad de la vía aérea
class AirwayDifficultyPredictor:
    def __init__(self):
        # Medias y desviaciones estándar estimadas para normalización
        self.means = {
            'Superficie lingual cm2': 24.32,
            'Distancia Piel a Epiglotis': 2.10,
            'Grosor de la lengua': 5.82,
            'Distancia Piel a Hueso Hioides': 1.11,
            'EDAD': 53.5,
            'IMC kg/m2': 26.85
        }

        self.stds = {
            'Superficie lingual cm2': 4.66,
            'Distancia Piel a Epiglotis': 0.47,
            'Grosor de la lengua': 0.78,
            'Distancia Piel a Hueso Hioides': 0.41,
            'EDAD': 17.75,
            'IMC kg/m2': 3.98
        }

        # Importancia de variables según el estudio
        self.importances = {
            'Superficie lingual cm2': 0.370635,
            'Distancia Piel a Epiglotis': 0.222797,
            'Grosor de la lengua': 0.159394,
            'Distancia Piel a Hueso Hioides': 0.117933,
            'IMC kg/m2': 0.074609,
            'EDAD': 0.054630
        }

        # Rangos aceptables para cada variable
        self.ranges = {
            'Superficie lingual cm2': (15.0, 33.63),
            'Distancia Piel a Epiglotis': (1.17, 3.03),
            'Grosor de la lengua': (4.27, 7.37),
            'Distancia Piel a Hueso Hioides': (0.3, 1.92),
            'EDAD': (18, 89),
            'IMC kg/m2': (18.9, 34.8)
        }

    def normalize_input(self, input_data):
        """Normaliza los datos de entrada usando z-score"""
        normalized = {}
        for key, value in input_data.items():
            normalized[key] = (value - self.means[key]) / self.stds[key]
        return normalized

    def validate_input(self, input_data):
        """Valida que los valores estén dentro de los rangos aceptables"""
        invalid_fields = []
        for key, (min_val, max_val) in self.ranges.items():
            if not min_val <= input_data[key] <= max_val:
                invalid_fields.append((key, min_val, max_val))
        return invalid_fields

    def predict(self, input_data):
        """Predice la dificultad de vía aérea basada en los datos de entrada"""
        # Normalizar los datos
        normalized_data = self.normalize_input(input_data)

        # Calcular score ponderado basado en importancias
        score = 0
        contribution = {}  # Para almacenar la contribución de cada variable

        for feature, importance in self.importances.items():
            # Calcular contribución según el tipo de variable
            if feature in ['Superficie lingual cm2', 'Grosor de la lengua', 'IMC kg/m2']:
                # Relación directa con dificultad
                contrib = normalized_data[feature] * importance
                score += contrib
            elif feature in ['Distancia Piel a Epiglotis', 'Distancia Piel a Hueso Hioides']:
                # Relación inversa (menor distancia, mayor dificultad)
                contrib = -normalized_data[feature] * importance
                score += contrib
            else:  # EDAD
                # Relación directa pero con menor peso
                contrib = normalized_data[feature] * importance * 0.5
                score += contrib

            contribution[feature] = contrib

        # Determinar clase según el score
        if score < -0.5:
            prediction = 0  # Fácil
            confidence = min(0.95, 0.75 - score)  # Mayor confianza mientras más negativo sea el score
        elif score < 1.0:
            prediction = 1  # Difícil
            # Calcular confianza según proximidad al centro del rango
            # Más cerca del centro = mayor confianza
            mid_range = 0.25  # Punto medio aproximado del rango (-0.5 a 1.0)
            distance_from_mid = abs(score - mid_range)
            confidence = max(0.65, 0.85 - distance_from_mid*0.2)
        else:
            prediction = 2  # Muy difícil
            confidence = min(0.95, 0.6 + score*0.15)  # Mayor confianza mientras más alto sea el score

        # Probabilidades aproximadas (suma = 1)
        if prediction == 0:
            probabilities = [confidence, (1-confidence)*0.8, (1-confidence)*0.2]
        elif prediction == 1:
            probabilities = [(1-confidence)*0.4, confidence, (1-confidence)*0.6]
        else:
            probabilities = [(1-confidence)*0.1, (1-confidence)*0.9, confidence]

        return {
            'prediction': prediction,
            'confidence': confidence,
            'probabilities': probabilities,
            'score': score,
            'contributions': contribution
        }

# Función para obtener un nombre amigable para la variable
def get_friendly_name(var_name):
    """Devuelve un nombre más amigable para la variable"""
    friendly_names = {
        'Superficie lingual cm2': 'Superficie lingual (cm²)',
        'Distancia Piel a Epiglotis': 'Distancia piel a epiglotis (cm)',
        'Grosor de la lengua': 'Grosor de la lengua (cm)',
        'Distancia Piel a Hueso Hioides': 'Distancia piel a hioides (cm)',
        'EDAD': 'Edad (años)',
        'IMC kg/m2': 'IMC (kg/m²)'
    }
    return friendly_names.get(var_name, var_name)

# Función para crear gráfico de importancia de variables
def plot_feature_importance(predictor):
    """Visualiza la importancia de las variables"""
    plt.figure(figsize=(10, 5))

    # Datos para el gráfico
    features = list(predictor.importances.keys())
    importances = list(predictor.importances.values())

    # Ordenar por importancia
    sorted_indices = np.argsort(importances)[::-1]  # Ordenar de mayor a menor
    sorted_features = [get_friendly_name(features[i]) for i in sorted_indices]
    sorted_importances = [importances[i] for i in sorted_indices]

    # Crear barras horizontales con gradiente
    colors = plt.cm.Blues(np.linspace(0.4, 0.9, len(features)))
    y_pos = np.arange(len(features))

    # Graficar barras horizontales
    bars = plt.barh(y_pos, sorted_importances, color=colors)

    # Etiquetas y aspecto
    plt.yticks(y_pos, sorted_features)
    plt.xlabel('Importancia Relativa')
    plt.title('Importancia de Variables en la Predicción', fontweight='bold')

    # Añadir valores en las barras
    for i, v in enumerate(sorted_importances):
        plt.text(v + 0.01, i, f'{v:.3f}', va='center', fontsize=9)

    # Ajustar diseño
    plt.tight_layout()
    plt.show()

# Función para graficar contribuciones
def plot_contributions(result):
    """Visualiza la contribución de cada variable al score final"""
    plt.figure(figsize=(10, 5))

    # Obtener contribuciones
    contributions = result['contributions']

    # Ordenar por magnitud absoluta de contribución
    features = list(contributions.keys())
    contrib_values = list(contributions.values())

    # Ordenar por contribución absoluta
    abs_contrib = [abs(c) for c in contrib_values]
    sorted_indices = np.argsort(abs_contrib)[::-1]  # Ordenar de mayor a menor
    sorted_features = [get_friendly_name(features[i]) for i in sorted_indices]
    sorted_contrib = [contrib_values[i] for i in sorted_indices]

    # Colores según si contribuye positiva o negativamente
    colors = ['#4CAF50' if c < 0 else '#E53935' for c in sorted_contrib]
    y_pos = np.arange(len(features))

    # Graficar barras horizontales
    bars = plt.barh(y_pos, sorted_contrib, color=colors)

    # Línea vertical en cero
    plt.axvline(x=0, color='black', linestyle='-', linewidth=0.5)

    # Etiquetas y aspecto
    plt.yticks(y_pos, sorted_features)
    plt.xlabel('Contribución al Score')
    plt.title('Influencia de cada Variable en este Paciente', fontweight='bold')

    # Añadir valores en las barras
    for i, v in enumerate(sorted_contrib):
        plt.text(
            v + (0.01 if v >= 0 else -0.01),
            i,
            f'{v:.2f}',
            va='center',
            ha='left' if v >= 0 else 'right',
            fontsize=9
        )

    # Añadir notas explicativas
    plt.text(
        0.98, 0.02,
        'Verde: Reduce dificultad\nRojo: Aumenta dificultad',
        transform=plt.gca().transAxes,
        ha='right', va='bottom',
        fontsize=9,
        bbox=dict(facecolor='white', alpha=0.8, edgecolor='none')
    )

    # Ajustar diseño
    plt.tight_layout()
    plt.show()

# Función para graficar probabilidades
def plot_probabilities(result):
    """Visualiza las probabilidades de cada clase"""
    plt.figure(figsize=(8, 5))

    # Datos para el gráfico
    labels = ['Fácil', 'Difícil', 'Muy difícil']
    probabilities = result['probabilities']

    # Colores para cada clase
    colors = ['#4CAF50', '#FF9800', '#E53935']

    # Graficar barras verticales
    bars = plt.bar(labels, probabilities, color=colors, width=0.6)

    # Resaltar la predicción principal
    prediction_idx = result['prediction']
    bars[prediction_idx].set_edgecolor('black')
    bars[prediction_idx].set_linewidth(2)

    # Etiquetas y aspecto
    plt.ylabel('Probabilidad')
    plt.ylim(0, 1)
    plt.title('Probabilidad de cada Clase', fontweight='bold')

    # Añadir porcentajes sobre las barras
    for i, v in enumerate(probabilities):
        plt.text(
            i, v + 0.02,
            f'{v*100:.1f}%',
            ha='center', va='bottom',
            fontsize=10
        )

    # Ajustar diseño
    plt.tight_layout()
    plt.show()

# Crear una calculadora interactiva con ipywidgets
def create_interactive_calculator():
    # Crear un predictor
    predictor = AirwayDifficultyPredictor()

    # Crear un estilo personalizado
    style = {'description_width': 'initial'}
    layout = {'width': '100%', 'margin': '0px 0px 10px 0px'}

    # Crear widgets para los parámetros
    superficie_lingual = widgets.FloatSlider(
        value=16.0,
        min=15.0,
        max=33.63,
        step=0.1,
        description='Superficie lingual (cm²):',
        style=style,
        layout=layout,
        readout_format='.1f'
    )

    distancia_piel_epiglotis = widgets.FloatSlider(
        value=2.0,
        min=1.17,
        max=3.03,
        step=0.01,
        description='Distancia piel a epiglotis (cm):',
        style=style,
        layout=layout,
        readout_format='.2f'
    )

    grosor_lengua = widgets.FloatSlider(
        value=6.0,
        min=4.27,
        max=7.37,
        step=0.01,
        description='Grosor de la lengua (cm):',
        style=style,
        layout=layout,
        readout_format='.2f'
    )

    distancia_piel_hioides = widgets.FloatSlider(
        value=1.0,
        min=0.3,
        max=1.92,
        step=0.01,
        description='Distancia piel a hioides (cm):',
        style=style,
        layout=layout,
        readout_format='.2f'
    )

    edad = widgets.IntSlider(
        value=75,
        min=18,
        max=89,
        step=1,
        description='Edad (años):',
        style=style,
        layout=layout
    )

    imc = widgets.FloatSlider(
        value=25.0,
        min=18.9,
        max=34.8,
        step=0.1,
        description='IMC (kg/m²):',
        style=style,
        layout=layout,
        readout_format='.1f'
    )

    # Botón de predicción
    predict_button = widgets.Button(
        description='Predecir',
        button_style='primary',
        layout=widgets.Layout(width='200px', margin='20px 0px')
    )

    # Área para mostrar resultados
    result_output = widgets.Output()
    graph_output = widgets.Output()

    # Mostrar la importancia de las variables
    with graph_output:
        display(HTML("<h3>Importancia de Variables en el Modelo</h3>"))
        plot_feature_importance(predictor)

    # Función para actualizar el resultado cuando se hace clic en el botón
    def on_predict_clicked(b):
        # Limpiar salidas anteriores
        result_output.clear_output()

        # Recopilar valores de entrada
        input_data = {
            'Superficie lingual cm2': superficie_lingual.value,
            'Distancia Piel a Epiglotis': distancia_piel_epiglotis.value,
            'Grosor de la lengua': grosor_lengua.value,
            'Distancia Piel a Hueso Hioides': distancia_piel_hioides.value,
            'EDAD': edad.value,
            'IMC kg/m2': imc.value
        }

        # Validar datos
        invalid_fields = predictor.validate_input(input_data)

        with result_output:
            if invalid_fields:
                display(HTML("<h3 style='color: #E53935;'>Valores fuera de rango:</h3>"))
                for field, min_val, max_val in invalid_fields:
                    display(HTML(f"<p>• {get_friendly_name(field)}: debe estar entre {min_val} y {max_val}</p>"))
                return

            # Realizar la predicción
            result = predictor.predict(input_data)

            # Determinar texto y color según la predicción
            difficulty_labels = {
                0: "Vía aérea fácil",
                1: "Vía aérea difícil (frova/guía flexible)",
                2: "Vía aérea muy difícil (pala hiperangulada/otro videolaringoscopio)"
            }

            difficulty_colors = {
                0: "#4CAF50",  # Verde
                1: "#FF9800",  # Naranja
                2: "#E53935"   # Rojo
            }

            prediction = result['prediction']
            confidence_pct = int(result['confidence'] * 100)

            # Mostrar resultado con estilo
            display(HTML(f"""
                <div style="padding: 15px; border-radius: 5px; background-color: #f8f9fa; margin-bottom: 20px;">
                    <h2 style="color: {difficulty_colors[prediction]};">Predicción: {difficulty_labels[prediction]}</h2>
                    <p style="font-size: 16px; margin-top: 10px;">Confianza: <strong>{confidence_pct}%</strong></p>
                    <p>Score calculado: {result['score']:.2f}</p>
                </div>
            """))

            # Mostrar gráficos
            display(HTML("<h3>Contribución de cada Variable al Resultado</h3>"))
            plot_contributions(result)

            display(HTML("<h3>Probabilidades por Clase</h3>"))
            plot_probabilities(result)

    # Conectar el botón a la función
    predict_button.on_click(on_predict_clicked)

    # Mostrar la interfaz
    display(HTML("<h1 style='color: #2c3e50; margin-bottom: 20px;'>Calculadora de Predicción de Vía Aérea Difícil</h1>"))
    display(HTML("<p style='margin-bottom: 15px;'>Ajuste los parámetros utilizando los deslizadores y haga clic en 'Predecir' para obtener el resultado.</p>"))

    # Organizar los controles
    display(HTML("<h2 style='color: #2c3e50; margin: 20px 0px 10px 0px;'>Parámetros del Paciente</h2>"))

    # Mostrar los widgets
    display(superficie_lingual)
    display(distancia_piel_epiglotis)
    display(grosor_lengua)
    display(distancia_piel_hioides)
    display(edad)
    display(imc)
    display(predict_button)
    display(result_output)
    display(graph_output)

# Ejecutar la calculadora interactiva
create_interactive_calculator()

FloatSlider(value=16.0, description='Superficie lingual (cm²):', layout=Layout(margin='0px 0px 10px 0px', widt…

FloatSlider(value=2.0, description='Distancia piel a epiglotis (cm):', layout=Layout(margin='0px 0px 10px 0px'…

FloatSlider(value=6.0, description='Grosor de la lengua (cm):', layout=Layout(margin='0px 0px 10px 0px', width…

FloatSlider(value=1.0, description='Distancia piel a hioides (cm):', layout=Layout(margin='0px 0px 10px 0px', …

IntSlider(value=75, description='Edad (años):', layout=Layout(margin='0px 0px 10px 0px', width='100%'), max=89…

FloatSlider(value=25.0, description='IMC (kg/m²):', layout=Layout(margin='0px 0px 10px 0px', width='100%'), ma…

Button(button_style='primary', description='Predecir', layout=Layout(margin='20px 0px', width='200px'), style=…

Output()

Output()

In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 2.6 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gradio as gr
import io
from PIL import Image

# Configuración de estilo
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_context("notebook", font_scale=1.2)

# Clase para predecir la dificultad de la vía aérea
class AirwayDifficultyPredictor:
    def __init__(self):
        # Medias y desviaciones estándar estimadas para normalización
        self.means = {
            'Superficie lingual cm2': 24.32,
            'Distancia Piel a Epiglotis': 2.10,
            'Grosor de la lengua': 5.82,
            'Distancia Piel a Hueso Hioides': 1.11,
            'EDAD': 53.5,
            'IMC kg/m2': 26.85
        }

        self.stds = {
            'Superficie lingual cm2': 4.66,
            'Distancia Piel a Epiglotis': 0.47,
            'Grosor de la lengua': 0.78,
            'Distancia Piel a Hueso Hioides': 0.41,
            'EDAD': 17.75,
            'IMC kg/m2': 3.98
        }

        # Importancia de variables según el estudio
        self.importances = {
            'Superficie lingual cm2': 0.370635,
            'Distancia Piel a Epiglotis': 0.222797,
            'Grosor de la lengua': 0.159394,
            'Distancia Piel a Hueso Hioides': 0.117933,
            'IMC kg/m2': 0.074609,
            'EDAD': 0.054630
        }

        # Rangos aceptables para cada variable
        self.ranges = {
            'Superficie lingual cm2': (15.0, 33.63),
            'Distancia Piel a Epiglotis': (1.17, 3.03),
            'Grosor de la lengua': (4.27, 7.37),
            'Distancia Piel a Hueso Hioides': (0.3, 1.92),
            'EDAD': (18, 89),
            'IMC kg/m2': (18.9, 34.8)
        }

    def normalize_input(self, input_data):
        """Normaliza los datos de entrada usando z-score"""
        normalized = {}
        for key, value in input_data.items():
            normalized[key] = (value - self.means[key]) / self.stds[key]
        return normalized

    def validate_input(self, input_data):
        """Valida que los valores estén dentro de los rangos aceptables"""
        invalid_fields = []
        for key, (min_val, max_val) in self.ranges.items():
            if not min_val <= input_data[key] <= max_val:
                invalid_fields.append((key, min_val, max_val))
        return invalid_fields

    def predict(self, input_data):
        """Predice la dificultad de vía aérea basada en los datos de entrada"""
        # Normalizar los datos
        normalized_data = self.normalize_input(input_data)

        # Calcular score ponderado basado en importancias
        score = 0
        contribution = {}  # Para almacenar la contribución de cada variable

        for feature, importance in self.importances.items():
            # Calcular contribución según el tipo de variable
            if feature in ['Superficie lingual cm2', 'Grosor de la lengua', 'IMC kg/m2']:
                # Relación directa con dificultad
                contrib = normalized_data[feature] * importance
                score += contrib
            elif feature in ['Distancia Piel a Epiglotis', 'Distancia Piel a Hueso Hioides']:
                # Relación inversa (menor distancia, mayor dificultad)
                contrib = -normalized_data[feature] * importance
                score += contrib
            else:  # EDAD
                # Relación directa pero con menor peso
                contrib = normalized_data[feature] * importance * 0.5
                score += contrib

            contribution[feature] = contrib

        # Determinar clase según el score
        if score < -0.5:
            prediction = 0  # Fácil
            confidence = min(0.95, 0.75 - score)  # Mayor confianza mientras más negativo sea el score
        elif score < 1.0:
            prediction = 1  # Difícil
            # Calcular confianza según proximidad al centro del rango
            # Más cerca del centro = mayor confianza
            mid_range = 0.25  # Punto medio aproximado del rango (-0.5 a 1.0)
            distance_from_mid = abs(score - mid_range)
            confidence = max(0.65, 0.85 - distance_from_mid*0.2)
        else:
            prediction = 2  # Muy difícil
            confidence = min(0.95, 0.6 + score*0.15)  # Mayor confianza mientras más alto sea el score

        # Probabilidades aproximadas (suma = 1)
        if prediction == 0:
            probabilities = [confidence, (1-confidence)*0.8, (1-confidence)*0.2]
        elif prediction == 1:
            probabilities = [(1-confidence)*0.4, confidence, (1-confidence)*0.6]
        else:
            probabilities = [(1-confidence)*0.1, (1-confidence)*0.9, confidence]

        return {
            'prediction': prediction,
            'confidence': confidence,
            'probabilities': probabilities,
            'score': score,
            'contributions': contribution
        }

# Función para obtener un nombre amigable para la variable
def get_friendly_name(var_name):
    """Devuelve un nombre más amigable para la variable"""
    friendly_names = {
        'Superficie lingual cm2': 'Superficie lingual (cm²)',
        'Distancia Piel a Epiglotis': 'Distancia piel a epiglotis (cm)',
        'Grosor de la lengua': 'Grosor de la lengua (cm)',
        'Distancia Piel a Hueso Hioides': 'Distancia piel a hioides (cm)',
        'EDAD': 'Edad (años)',
        'IMC kg/m2': 'IMC (kg/m²)'
    }
    return friendly_names.get(var_name, var_name)

# Función para crear gráfico de contribuciones y devolverlo como imagen
def plot_contributions_image(result):
    """Visualiza la contribución de cada variable al score final y devuelve como imagen"""
    plt.figure(figsize=(10, 5))

    # Obtener contribuciones
    contributions = result['contributions']

    # Ordenar por magnitud absoluta de contribución
    features = list(contributions.keys())
    contrib_values = list(contributions.values())

    # Ordenar por contribución absoluta
    abs_contrib = [abs(c) for c in contrib_values]
    sorted_indices = np.argsort(abs_contrib)[::-1]  # Ordenar de mayor a menor
    sorted_features = [get_friendly_name(features[i]) for i in sorted_indices]
    sorted_contrib = [contrib_values[i] for i in sorted_indices]

    # Colores según si contribuye positiva o negativamente
    colors = ['#4CAF50' if c < 0 else '#E53935' for c in sorted_contrib]
    y_pos = np.arange(len(features))

    # Graficar barras horizontales
    bars = plt.barh(y_pos, sorted_contrib, color=colors)

    # Línea vertical en cero
    plt.axvline(x=0, color='black', linestyle='-', linewidth=0.5)

    # Etiquetas y aspecto
    plt.yticks(y_pos, sorted_features)
    plt.xlabel('Contribución al Score')
    plt.title('Influencia de cada Variable en este Paciente', fontweight='bold')

    # Añadir valores en las barras
    for i, v in enumerate(sorted_contrib):
        plt.text(
            v + (0.01 if v >= 0 else -0.01),
            i,
            f'{v:.2f}',
            va='center',
            ha='left' if v >= 0 else 'right',
            fontsize=9
        )

    # Añadir notas explicativas
    plt.text(
        0.98, 0.02,
        'Verde: Reduce dificultad\nRojo: Aumenta dificultad',
        transform=plt.gca().transAxes,
        ha='right', va='bottom',
        fontsize=9,
        bbox=dict(facecolor='white', alpha=0.8, edgecolor='none')
    )

    # Ajustar diseño
    plt.tight_layout()

    # Guardar la figura en un objeto BytesIO
    buf = io.BytesIO()
    plt.savefig(buf, format='png', dpi=100)
    buf.seek(0)

    # Convertir el objeto BytesIO a una imagen PIL
    img = Image.open(buf)

    # Cerrar la figura para liberar memoria
    plt.close()

    return img

# Función para crear gráfico de probabilidades y devolverlo como imagen
def plot_probabilities_image(result):
    """Visualiza las probabilidades de cada clase y devuelve como imagen"""
    plt.figure(figsize=(8, 5))

    # Datos para el gráfico
    labels = ['Fácil', 'Difícil', 'Muy difícil']
    probabilities = result['probabilities']

    # Colores para cada clase
    colors = ['#4CAF50', '#FF9800', '#E53935']

    # Graficar barras verticales
    bars = plt.bar(labels, probabilities, color=colors, width=0.6)

    # Resaltar la predicción principal
    prediction_idx = result['prediction']
    bars[prediction_idx].set_edgecolor('black')
    bars[prediction_idx].set_linewidth(2)

    # Etiquetas y aspecto
    plt.ylabel('Probabilidad')
    plt.ylim(0, 1)
    plt.title('Probabilidad de cada Clase', fontweight='bold')

    # Añadir porcentajes sobre las barras
    for i, v in enumerate(probabilities):
        plt.text(
            i, v + 0.02,
            f'{v*100:.1f}%',
            ha='center', va='bottom',
            fontsize=10
        )

    # Ajustar diseño
    plt.tight_layout()

    # Guardar la figura en un objeto BytesIO
    buf = io.BytesIO()
    plt.savefig(buf, format='png', dpi=100)
    buf.seek(0)

    # Convertir el objeto BytesIO a una imagen PIL
    img = Image.open(buf)

    # Cerrar la figura para liberar memoria
    plt.close()

    return img

# Función de predicción para Gradio
def predict_airway_difficulty(
    superficie_lingual,
    distancia_piel_epiglotis,
    grosor_lengua,
    distancia_piel_hioides,
    edad,
    imc
):
    predictor = AirwayDifficultyPredictor()

    # Recopilar los valores de entrada
    input_data = {
        'Superficie lingual cm2': superficie_lingual,
        'Distancia Piel a Epiglotis': distancia_piel_epiglotis,
        'Grosor de la lengua': grosor_lengua,
        'Distancia Piel a Hueso Hioides': distancia_piel_hioides,
        'EDAD': edad,
        'IMC kg/m2': imc
    }

    # Validar que los valores estén dentro de los rangos
    invalid_fields = predictor.validate_input(input_data)
    if invalid_fields:
        error_msg = "Valores fuera de rango:\n"
        for field, min_val, max_val in invalid_fields:
            error_msg += f"- {field}: debe estar entre {min_val} y {max_val}\n"
        return error_msg, None, None

    # Predecir la dificultad
    result = predictor.predict(input_data)

    # Determinar el texto según la predicción
    difficulty_labels = {
        0: "Predicción: Vía aérea fácil",
        1: "Predicción: Vía aérea difícil (frova/guía flexible)",
        2: "Predicción: Vía aérea muy difícil (pala hiperangulada/otro videolaringoscopio)"
    }

    prediction = result['prediction']
    confidence_pct = int(result['confidence'] * 100)

    # Crear mensajes de resultado
    prediction_text = f"{difficulty_labels[prediction]} (Confianza: {confidence_pct}%)"

    # Crear imágenes de los gráficos
    contribution_img = plot_contributions_image(result)
    probability_img = plot_probabilities_image(result)

    return prediction_text, contribution_img, probability_img

# Crear la interfaz Gradio
with gr.Blocks(css="footer {visibility: hidden}") as demo:
    gr.Markdown(
        """
        # Calculadora de Predicción de Vía Aérea Difícil

        Esta aplicación predice la dificultad de intubación basada en mediciones anatómicas y datos del paciente.
        """
    )

    with gr.Row():
        with gr.Column(scale=1):
            # Inputs
            superficie_lingual = gr.Slider(
                minimum=15.0, maximum=33.63, step=0.1, value=16.0,
                label="Superficie lingual (cm²)"
            )
            distancia_piel_epiglotis = gr.Slider(
                minimum=1.17, maximum=3.03, step=0.01, value=2.0,
                label="Distancia Piel a Epiglotis (cm)"
            )
            grosor_lengua = gr.Slider(
                minimum=4.27, maximum=7.37, step=0.01, value=6.0,
                label="Grosor de la lengua (cm)"
            )
            distancia_piel_hioides = gr.Slider(
                minimum=0.3, maximum=1.92, step=0.01, value=1.0,
                label="Distancia Piel a Hueso Hioides (cm)"
            )
            edad = gr.Slider(
                minimum=18, maximum=89, step=1, value=75,
                label="Edad (años)"
            )
            imc = gr.Slider(
                minimum=18.9, maximum=34.8, step=0.1, value=25.0,
                label="IMC (kg/m²)"
            )

            # Botón de predicción
            predict_btn = gr.Button("Predecir", variant="primary")

        with gr.Column(scale=1):
            # Outputs
            prediction_text = gr.Textbox(label="Resultado de la Predicción")
            contribution_plot = gr.Image(label="Contribución de Variables", type="pil")
            probability_plot = gr.Image(label="Probabilidad por Clase", type="pil")

    # Explicación de la importancia de variables
    gr.Markdown(
        """
        ## Información sobre las Variables

        Las variables están ordenadas por importancia según el modelo:
        1. Superficie lingual (cm²): 0.371
        2. Distancia Piel a Epiglotis (cm): 0.223
        3. Grosor de la lengua (cm): 0.159
        4. Distancia Piel a Hueso Hioides (cm): 0.118
        5. IMC (kg/m²): 0.075
        6. Edad (años): 0.055
        """
    )

    # Conectar los inputs y outputs con la función de predicción
    predict_btn.click(
        predict_airway_difficulty,
        inputs=[
            superficie_lingual,
            distancia_piel_epiglotis,
            grosor_lengua,
            distancia_piel_hioides,
            edad,
            imc
        ],
        outputs=[
            prediction_text,
            contribution_plot,
            probability_plot
        ]
    )

# Iniciar la aplicación
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b0040c37f80666f6db.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
